# intro to Smartcore

Let's start installing the library dependency.

In [2]:
// cargo install the newest version
// this may take a while, needs to download and install the library
:dep smartcore = "0.3.0"

As an example of what Smartcore provides to the users, let's try `sigmoid` provided by the new `RealNumber` trait:

In [3]:
use smartcore::numbers::realnum::*;   // this line is not needed, just to be explicit

println!("{:?}", 1.0.sigmoid());      // 0.7310585786300049
println!("{:?}", 41.0.sigmoid());     // 1.
println!("{:?}", (-41.0).sigmoid());  // 0


0.7310585786300049
1.0
0.0


## -> where to start

*If you want to just jump into running algorithms on your data*, the best place to start is instantiating a `smartcore::linalg::basic::matrix::DenseMatrix`; with this handy structure you can leverage all the functionalities provided by the traits system with a simple interface. For the usage of  `DenseMatrix` see the other notebooks where procedures are presented.

*If you are interested in understanding how Smartcore works* because you are thinking about contributing or you are just curious, keep reading here and you will find some nuggets.

## numbers types

As you can see `RealNumber` extends the usual `float` with some convenience methods. Every time the function `sigmoid` is called on a Rust "numeric" type, the type system reconciles the traits attached to that type to find the function. Obviously an error is presented by the wonderful Rust compiler if no method can be found or if there is ambiguity between types providing the same method. 

All the `numbers` traits implement some of Rust standard library's primitives with usefull methods, like `sigmoid`.

`smartcore::numbers::basenum::Number` is implemented for all Rust's primitives:
```
impl Number for f64 {}
impl Number for f32 {}
impl Number for i8 {}
impl Number for i16 {}
impl Number for i32 {}
impl Number for i64 {}
impl Number for u8 {}
impl Number for u16 {}
impl Number for u32 {}
impl Number for u64 {}
impl Number for usize {}
```

Smartcore `numbers` provides traits that are composition of [`num-traits` library's](https://docs.rs/num-traits/latest/num_traits/) base traits:
* `basenum::Number`: . Composed by traits like `Bounded` and `Copy`, you can look up all the ones that are part of the set.
* `floatnum::FloatNumber`: a trait that is the composition of `Number + num_traits::Float + num_traits::Signed`
* `realnum::RealNumber`: a trait to specify Reals (Rational and Irrational), it is the composition of `Number + Float`

This provides useful constraints like for example: only `Number` allows absolute ordering (`Ord`), `FloatNumber` and `RealNumber` are required `PartialEq` for evident reasons, and all the nice constraints necessary to leverage a type system. 

This powerful representation is hidden to the end-user under useful structures like, again, `DenseMatrix` (`SparseMatrix` to be there soon enough). Whatever Rust primitive (strings will also be implemented soon enough) you decide to use in your matrices the traits system will check that the constraints are satisfied, making everything safer and providing compile-time goodies. 

The traits system is not limited to these foundamental traits, you will see more theoretically-defined traits applied to matrices like the ones in `smartcore::linalg::traits` that allow properties to be attached to 2D arrays with particular characteristics. 


## vectors (1D arrays)

Smartcore also provides useful extensions for Rust's `Vec`, you can find those in `src/linalg/basic/vector.rs`. For example this implementation:
```
impl<T: Debug + Display + Copy + Sized> Array1<T> for Vec<T> { ... }
```
extends a generic `Vec<T>` with Smartcore's custom capabilities from `smartcore::lianalg::basic::arrays::Array1`, like creating a mutable instance from a `Range` (slice):
```
fn slice_mut<'b>(&'b mut self, range: Range<usize>) -> Box<dyn MutArrayView1<T> + 'b>
```

Other interesting convenience methods provided by this module:
* `from_iterator`
* `iterator_mut`
* `from_vec_slice`
* ...

Linear Algebra operations are collected in:
```
pub trait MutArray<T: Debug + Display + Copy + Sized, S>: Array<T, S> { ... }
```

Let's see some examples leveraging mutability:


In [12]:
// getting an element

// we  need this trait in scope to handle mutable arrays
use smartcore::linalg::basic::arrays::MutArray;

// this is a mutable Vec<i32>
let mut x = vec![1, 2, 3];

// this unwraps the Option<&T> and prints the value
println!("original value:\n\t{:?}", *x.get(2));

// this sets the value at position 1
x.set(1, 101);

println!("set value:\n\t{:?}", x.get(1));

original value:
	3
set value:
	101


In [17]:
// we  need this trait in scope to use iterator
use smartcore::linalg::basic::arrays::Array;

// iterate over one of the axis of the array
// use `Array::iterator` on a Vec<i32>
let v: Vec<i32> = vec![1, 2, 3];
let mut pow_: Vec<i32> = vec![];
v.iterator(0).for_each(|v| pow_.push(v.pow(2)));
println!("initial vector:\n\t{:?}", v);
println!("power of 2 of vector:\n\t{:?}", pow_);

// same but with mutability
// use `Array::iterator_mut` on a Vec<i32>
println!("This iterator changes sign to its elements");
vec![1, 2, 3].iterator_mut(0)   // instantiate a mutable iterator on axis 0
    .for_each(|v| {
        *v = -*v;                         // change sign to all the elements
        println!("\t{:?}", v.clone());
    }
);                


// use of axis
// is this the right behaviour? should we return a `Result` instead?
println!("This line panics as an `Array` has no second axis:");
let _ = vec![1, 2, 3].iterator(1);

initial vector:
	[1, 2, 3]
power of 2 of vector:
	[1, 4, 9]
This iterator changes sign to its elements
	-1
	-2
	-3
This line panics as an `Array` has no second axis:


thread '<unnamed>' panicked at 'For one dimensional array `axis` should == 0', /home/lorenzo/.cargo/git/checkouts/smartcore-91f9710bf445643e/42d017e/src/linalg/basic/vector.rs:32:9
stack backtrace:
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [5]:
use smartcore::linalg::basic::arrays::Array1;  // `argsort` trait method needed in scope

let arr2 = vec![
    0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2, 0.2, 0.1, 1.4, 1.5, 1.5, 1.3, 1.5, 1.3, 1.6,
    1.0, 1.3, 1.4,
];

println!("argsort()\n\t{:?}",
    arr2.argsort()                            // returns ordered indices
);

argsort()
	[9, 7, 1, 8, 0, 2, 4, 3, 6, 5, 17, 18, 15, 13, 19, 10, 14, 11, 12, 16]


In [6]:
println!("take(&slice)\n\t{:?}",
    vec![1, 2, 3, 4, 5, 6].take(&[0, 4, 5])
);

take(&slice)
	[1, 5, 6]


In [7]:
use smartcore::linalg::basic::arrays::MutArrayView1;  // `copy_from` trait method needed in scope

let x = vec![1, 2, 3];
let mut y = Vec::<i32>::zeros(3);
y.copy_from(&x);
println!("y.copy_from(&x)\n\t{:?}",
    y
);

y.copy_from(&x)
	[1, 2, 3]


## matrix (2D arrays)

The great potential of Rust's traits system is that the above can be easily applied to higher-dimensional arrays than vectors. Smartcore provides `smartcore::linalg::basic::arrays::Array2` as a trait to define matrices, the one used to define `DenseMatrix`.

These other methods are provided by `Array2` in addition to the ones in `MutArray` (that is itself a composition of `Array`). just to mention few available methods:
* `get_row(..., row: usize)`
* `get_column(..., row: usize)`
* `eye(size: usize)`
* `from_row(slice: ...)`
* `from_columns(slice: ...)`
* `transpose()`
* `reshape(..., nrows: usize, ncols: usize, axis: u8)`
* `ab(&self, a_transpose: bool, b: &dyn ArrayView2<T>, b_transpose: bool)`
* `ax(&self, a_transpose: bool, x: &dyn ArrayView1<T>)`
* `concatenate_2d(arrays: &'a [&'a dyn ArrayView2<T>], axis: u8)`
* ...

These are added to already existing computational methods that apply to specific `axis` in the structures, like:
* `norm(..., p: f64(`
* `cov(..., cov: &mut dyn MutArrayView2<f64>)`
* `std()`
* `var()`
* `argmax()`
* ...

In [8]:
use smartcore::linalg::basic::arrays::{Array, Array2, ArrayView2, MutArrayView2};
use smartcore::linalg::basic::matrix::DenseMatrix;

let x = DenseMatrix::from_2d_array(&[&[1, 2, 3], &[4, 5, 6]]);
println!("transposed\n\t{:?}",
    x.transpose()
);

println!("from_slice\n\t{:?}",
    DenseMatrix::from_slice(x.slice(0..2, 0..2).as_ref())
);

println!("from_row\n\t{:?}",
    DenseMatrix::from_row(x.get_row(0).as_ref())
);

println!("from_column\n\t{:?}",
    DenseMatrix::from_column(x.get_col(0).as_ref())
);

println!("zeros(2, 2)\n\t{:?}",
    DenseMatrix::<i32>::zeros(2, 2)
);

println!("eye(3)\n\t{:?}",
    DenseMatrix::<i32>::eye(3)
);

transposed
	DenseMatrix { ncols: 2, nrows: 3, values: [1, 4, 2, 5, 3, 6], column_major: false }
from_slice
	DenseMatrix { ncols: 2, nrows: 2, values: [1, 2, 4, 5], column_major: false }
from_row
	DenseMatrix { ncols: 3, nrows: 1, values: [1, 2, 3], column_major: false }
from_column
	DenseMatrix { ncols: 1, nrows: 2, values: [1, 4], column_major: false }
zeros(2, 2)
	DenseMatrix { ncols: 2, nrows: 2, values: [0, 0, 0, 0], column_major: true }
eye(3)
	DenseMatrix { ncols: 3, nrows: 3, values: [1, 0, 0, 0, 1, 0, 0, 0, 1], column_major: true }


In [9]:
println!("from_2d_array and argmax(0)\n\t{:?}",
    DenseMatrix::from_2d_array(&[&[1, 5, 3], &[4, 2, 6]]).argmax(0)  // `argmax` on axis 0 (columns)
);

println!("from_2d_array and argmax(1)\n\t{:?}",
    DenseMatrix::from_2d_array(&[&[1, 5, 3], &[4, 2, 6]]).argmax(1)  // `argmax` on axis 1 (rows)
);

from_2d_array and argmax(0)
	[1, 0, 1]
from_2d_array and argmax(1)
	[1, 2]


In [10]:
// using partial equality for matrices
let x = DenseMatrix::from_2d_array(&[&[1, 2, 3], &[4, 5, 6]]);
let y = DenseMatrix::from_2d_array(&[&[1, 2, 3], &[4, 5, 6]]);

println!("x equal to y\n\t{:?}",
   x.eq(&y)
);

let x2 = DenseMatrix::from_2d_array(&[&[1, 2, 3], &[4, 5, 6]]);
let y2 = DenseMatrix::from_2d_array(&[&[1, 2, 3], &[4, 5, 7]]);

println!("x2 equal to y2\n\t{:?}",
   x2.eq(&y2)
);

x equal to y
	true
x2 equal to y2
	false


In [11]:
// leverage the `approx` library
:dep approx = "0.5.1"

use approx::AbsDiffEq;  // need to import trait in scope for Absolute Difference Equality

let x2_float = DenseMatrix::from_2d_array(&[&[1., 2.3, 3.], &[4., 5., 6.2]]);
let y2_float = DenseMatrix::from_2d_array(&[&[1., 2.6, 3.], &[4., 5., 6.1]]);

println!("x2_float equal to y2_float with a difference (epsilon) of 0.5\n\t{:?}",
    x2_float.abs_diff_eq(&y2_float, 0.5)
);

x2_float equal to y2_float with a difference (epsilon) of 0.5
	true
